In [21]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Input, Add, Reshape,Conv2D,MaxPooling2D,ZeroPadding2D,experimental,AveragePooling2D,Concatenate,Dropout
from matplotlib import pyplot as plt
from tensorflow.keras.models import Model
import numpy as np
import keras
from keras import layers
from PIL import Image

ImportError: cannot import name 'Resizing' from 'tensorflow.keras.layers' (C:\Users\DeLL\Anaconda3\lib\site-packages\tensorflow\python\keras\api\_v2\keras\layers\__init__.py)

In [42]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [4]:
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [43]:
Y_test = tf.keras.utils.to_categorical(y_test)

In [25]:
X_train = []
for i,img in enumerate(x_train):
  X_train.append(np.array(Image.fromarray(img).convert('L').resize((34,34),Image.BILINEAR)))

In [26]:
X_train = np.array(X_train)
X_train = np.expand_dims(X_train,axis = -1)

In [39]:
i = tf.keras.layers.Input([32, 32, 3], dtype = tf.uint8)
x = tf.cast(i, tf.float32)
x = tf.keras.applications.densenet.preprocess_input(x)
core = tf.keras.applications.DenseNet121( include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=10)
x = core.layers[5](x)
model = tf.keras.Model(inputs=[i], outputs=[x])

In [8]:
syn = []
for img in x_train[:10000]:
  img = np.expand_dims(img,axis = 0)
  filters = np.squeeze(model(img),axis = 0)
  #syn.append(filters)
  for j in range(3):
    syn.append(filters[:,:,j])

In [9]:
labels = []
for i in y_train[:10000]:
    for j in range(3):
      labels.append(i)

In [17]:
labels = np.array(labels)
data   = np.array(syn)
data = np.expand_dims(data,axis = -1)

In [40]:
#input = Input((34,34,1))
c1l1  = Conv2D(32,3,(1,1),padding = 'same',activation='relu')(x)
c1l2  = Conv2D(32,3,(1,1),padding = 'same',activation='relu')(c1l1)
ces2  = Add()([c1l2, c1l1])
c1l3  = Conv2D(32,3,(1,1),padding = 'same',activation='relu')(ces2)
ces3  = Add()([c1l3, ces2])
ces3  = Flatten()(ces3)
ces3  = Dropout(0.5)(ces3)
ces3  = Dense(10,activation='softmax')(ces3)
model = Model(inputs=i, outputs=ces3)

In [41]:
batch_size = 32
#callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=50,shuffle = True, validation_split=0.1)#,callbacks=[callback])

Train on 45000 samples, validate on 5000 samples
Epoch 1/50
45000/45000 [==============================] - 339s 8ms/sample - loss: 1.4392 - accuracy: 0.5236 - val_loss: 1.1117 - val_accuracy: 0.6120
Epoch 2/50
45000/45000 [==============================] - 232s 5ms/sample - loss: 0.9723 - accuracy: 0.6646 - val_loss: 1.0844 - val_accuracy: 0.6330
Epoch 3/50
45000/45000 [==============================] - 225s 5ms/sample - loss: 0.8035 - accuracy: 0.7218 - val_loss: 1.0933 - val_accuracy: 0.6432
Epoch 4/50
45000/45000 [==============================] - 241s 5ms/sample - loss: 0.6563 - accuracy: 0.7738 - val_loss: 1.1787 - val_accuracy: 0.6450
Epoch 5/50
45000/45000 [==============================] - 227s 5ms/sample - loss: 0.5144 - accuracy: 0.8222 - val_loss: 1.1778 - val_accuracy: 0.6586
Epoch 6/50
45000/45000 [==============================] - 220s 5ms/sample - loss: 0.4003 - accuracy: 0.8633 - val_loss: 1.3852 - val_accuracy: 0.6404
Epoch 7/50
45000/45000 [===========================

In [44]:
result = []
for i in range(10):
  pos,_ = np.where(y_test == i)
  result.append(model.evaluate(x_test[pos],Y_test[pos],batch_size=100))
'''
airplane
automobile
bird 
cat
deer
dog
frog
horse
ship
truck
'''

1000/1000 [==============================] - 1s 1ms/sample - loss: 3.6370 - accuracy: 0.6910


'\nairplane\nautomobile\nbird \ncat\ndeer\ndog\nfrog\nhorse\nship\ntruck\n'

In [45]:
model.evaluate(x_test,Y_test)

10000/10000 [==============================] - 12s 1ms/sample - loss: 4.6140 - accuracy: 0.6047


[4.614019118499756, 0.6047]

In [48]:
x = .6080+.7750+.5600+0.4170+0.5250+0.4160+0.6850+0.5920+0.7000+0.6970

In [49]:
x

5.975

In [ ]:
"""
10% accuracy on validation, loss remain 14.

"""

In [50]:
model.layers